In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras_metrics import binary_average_recall, binary_recall, binary_precision
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/nn_training_data.csv", sep="|")
data.head()

,Unnamed: 0,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,totalScannedLineItems,lsvc_predict,lsvc_proba,xgb_predict,xgb_proba,fraud
0,0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,29.0,0,1.0,0,1.000,0
1,1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,14.0,0,1.0,0,1.000,0
2,2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,13.0,0,1.0,0,1.000,0
3,3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,29.0,0,1.0,0,0.928,0
4,4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,27.0,0,1.0,0,1.000,0


In [3]:
X = data[["lsvc_predict", "lsvc_proba", "xgb_predict", "xgb_proba"]]
y = data.fraud

In [4]:
model = Sequential()
model.add(Dense(15, input_shape=(4,),activation='relu'))
model.add(Dense(1,activation='sigmoid'))


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                75        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy',binary_precision(), binary_recall(), binary_average_recall()])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)


In [8]:
from keras.callbacks import TensorBoard, EarlyStopping
tensorboard = TensorBoard(log_dir="../logs")
early_stopping = EarlyStopping(monitor='val_precision',min_delta=1e-10, mode=max)

/home/lukas/Projects/dmc2019/venv/lib/python3.5/site-packages/keras/callbacks.py:511: RuntimeWarning: EarlyStopping mode <built-in function max> is unknown, fallback to auto mode.
  RuntimeWarning)


In [9]:
model.fit(X_train, y_train, batch_size=1000, epochs=500, validation_data=(X_test,y_test), callbacks=[tensorboard])

Instructions for updating:
Use tf.cast instead.
Train on 1409 samples, validate on 470 samples
Epoch 1/500
1409/1409 [==============================] - 0s 99us/step - loss: 0.8260 - acc: 0.0561 - precision: 0.0561 - recall: 1.0000 - average_recall: 1.0000 - val_loss: 0.8170 - val_acc: 0.0532 - val_precision: 0.0532 - val_recall: 1.0000 - val_average_recall: 1.0000
Epoch 2/500
1409/1409 [==============================] - 0s 5us/step - loss: 0.8152 - acc: 0.0561 - precision: 0.0561 - recall: 1.0000 - average_recall: 1.0000 - val_loss: 0.8062 - val_acc: 0.0532 - val_precision: 0.0532 - val_recall: 1.0000 - val_average_recall: 1.0000
Epoch 3/500
1409/1409 [==============================] - 0s 4us/step - loss: 0.8047 - acc: 0.0561 - precision: 0.0561 - recall: 1.0000 - average_recall: 1.0000 - val_loss: 0.7956 - val_acc: 0.0532 - val_precision: 0.0532 - val_recall: 1.0000 - val_average_recall: 1.0000
Epoch 4/500
1409/1409 [==============================] - 0s 4us/step - loss: 0.7941 - acc: 

In [10]:
model.save_weights("../data/nn_weights.h5")